In [ ]:
# 필요한 패키지 설치
!apt-get install -y wkhtmltopdf  # wkhtmltopdf 설치
!pip install nbformat nbconvert  # nbformat과 nbconvert 설치

# 이후에 제공된 코드 실행
from google.colab import files
import nbformat
from nbconvert import HTMLExporter
from subprocess import run
import os

# ipynb 파일 업로드
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# ipynb 파일 열기
with open(file_name, "r", encoding="utf-8") as f:
    notebook_content = nbformat.read(f, as_version=4)

# HTML 변환기 설정
html_exporter = HTMLExporter()
html_exporter.exclude_input = False

# 추가할 CSS 스타일 정의 (컴팩트한 레이아웃과 균등 여백 설정)
custom_css = """
<style>
    /* 페이지 여백 조정 */
    body {
        margin: 0;
        padding: 10px;
    }
    /* 이미지와 출력 셀 크기를 줄임 */
    img, .output {
        max-width: 90%;
        height: auto;
    }
    /* 코드 셀 및 텍스트 셀 간격 축소 */
    .input, .output {
        margin: 5px 0;
        padding: 5px;
        font-size: 12px;
    }
</style>
"""

# HTML 변환
html_data, _ = html_exporter.from_notebook_node(notebook_content)
html_data = custom_css + html_data  # CSS 추가

# HTML 파일 저장
html_file_name = file_name.replace(".ipynb", ".html")
with open(html_file_name, "w", encoding="utf-8") as html_file:
    html_file.write(html_data)

# HTML을 PDF로 변환 (상하좌우 여백을 25mm로 설정)
pdf_file_name = file_name.replace(".ipynb", ".pdf")
run([
    "wkhtmltopdf",
    "--margin-top", "25mm",
    "--margin-bottom", "25mm",
    "--margin-left", "25mm",
    "--margin-right", "25mm",
    "--page-size", "A4",
    html_file_name,
    pdf_file_name
])

# PDF 파일 다운로드
files.download(pdf_file_name)
print(f"PDF 파일이 생성되었습니다: {pdf_file_name}")